In [82]:
# Ελεγχος version pyspark .. Θέλουμε το 3.4.1
import pyspark
print(pyspark.__version__)

3.4.1


In [83]:
# add java to path .. δεν μπορουσα να βρω αλλον τροπο να τρέξει η spark.
# (θελουμε την Java 17 επειδη μονο η Java 8 , Java 11 και Java 17 δουλευει με το pyspark)
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"
os.environ["PATH"] += ";C:\\Program Files\\Java\\jdk-17\\bin"

In [108]:
from pyspark.sql import SparkSession
import json
from pyspark.sql.functions import col, avg, month, stddev, date_format
from pyspark.sql.types import TimestampType, FloatType


In [85]:
# Φτιάχνουμε ενα spark session
spark = SparkSession.builder.appName("TemperatureAnalysis").getOrCreate()

In [86]:
# Μετατρέπω τα text αρχεία σε json για να μπορέσω να τα διαβάσω με το spark
# Επειδή δεν βρήκαμε άλλον τρόπο να τα διαβάσουμε ως text αρχεία
convert = [["data/tempm.txt", "data/tempm.json", "temperature"], ["data/hum.txt", "data/hum.json" , "humidity"]]
for i,j,string in convert:
    with open(i, 'r') as input_file, open(j, 'w') as output_file:
        for line in input_file:
            data = json.loads(line)
            new_data = [{"timestamp": key, string: value} for key, value in data.items()]
            json.dump(new_data, output_file)
            output_file.write('\n')

tempm_data = spark.read.json("data/tempm.json")
hum_data = spark.read.json("data/hum.json")

In [87]:
# Ξεχωρίζω την ημερομινία απο το timestamp με την εντολή cast της pyspark

tempm_data = tempm_data.withColumn("timestamp", col("timestamp").cast(TimestampType()))
hum_data = hum_data.withColumn("timestamp", col("timestamp").cast(TimestampType()))

tempm_data = tempm_data.withColumn("temperature", col("temperature").cast(FloatType()))
hum_data = hum_data.withColumn("humidity", col("humidity").cast(FloatType()))

tempm_data = tempm_data.withColumn("date", col("timestamp").cast("date"))
hum_data = hum_data.withColumn("date", col("timestamp").cast("date"))


In [88]:
# Ελέγχω αν διαβάστηκαν σωστά τα δεδομένα
tempm_data.show()

+-----------+-------------------+----------+
|temperature|          timestamp|      date|
+-----------+-------------------+----------+
|        3.0|2014-02-13 06:20:00|2014-02-13|
|        7.0|2014-02-13 13:50:00|2014-02-13|
|        2.0|2014-02-13 06:00:00|2014-02-13|
|        3.0|2014-02-13 03:00:00|2014-02-13|
|        6.0|2014-02-13 13:00:00|2014-02-13|
|        4.0|2014-02-13 18:50:00|2014-02-13|
|        6.0|2014-02-13 13:20:00|2014-02-13|
|        6.0|2014-02-13 15:00:00|2014-02-13|
|        4.0|2014-02-13 08:50:00|2014-02-13|
|        4.0|2014-02-13 21:50:00|2014-02-13|
|        3.0|2014-02-13 08:00:00|2014-02-13|
|        3.0|2014-02-13 07:50:00|2014-02-13|
|        4.0|2014-02-13 08:20:00|2014-02-13|
|        3.0|2014-02-13 21:20:00|2014-02-13|
|        6.0|2014-02-13 11:50:00|2014-02-13|
|        6.0|2014-02-13 11:20:00|2014-02-13|
|        5.0|2014-02-13 17:50:00|2014-02-13|
|        6.0|2014-02-13 11:00:00|2014-02-13|
|        2.0|2014-02-13 05:50:00|2014-02-13|
|        3

In [89]:
# Ελέγχω αν διαβάστηκαν σωστά τα δεδομένα
hum_data.show()

+--------+-------------------+----------+
|humidity|          timestamp|      date|
+--------+-------------------+----------+
|    93.0|2014-02-13 06:20:00|2014-02-13|
|    66.0|2014-02-13 13:50:00|2014-02-13|
|    91.0|2014-02-13 06:00:00|2014-02-13|
|    84.0|2014-02-13 03:00:00|2014-02-13|
|    62.0|2014-02-13 13:00:00|2014-02-13|
|    75.0|2014-02-13 18:50:00|2014-02-13|
|    70.0|2014-02-13 13:20:00|2014-02-13|
|    56.0|2014-02-13 15:00:00|2014-02-13|
|    87.0|2014-02-13 08:50:00|2014-02-13|
|    75.0|2014-02-13 21:50:00|2014-02-13|
|    88.0|2014-02-13 08:00:00|2014-02-13|
|    93.0|2014-02-13 07:50:00|2014-02-13|
|    87.0|2014-02-13 08:20:00|2014-02-13|
|    81.0|2014-02-13 21:20:00|2014-02-13|
|    76.0|2014-02-13 11:50:00|2014-02-13|
|    76.0|2014-02-13 11:20:00|2014-02-13|
|    70.0|2014-02-13 17:50:00|2014-02-13|
|    69.0|2014-02-13 11:00:00|2014-02-13|
|    93.0|2014-02-13 05:50:00|2014-02-13|
|    81.0|2014-02-13 20:50:00|2014-02-13|
+--------+-------------------+----

# Ερώτημα 1.1

In [90]:
# Φιλτράρω την θερμοκρασία μεταξύ 18°C και 22°C
filtered_temp = tempm_data.filter((col("temperature") >= 18) & (col("temperature") <= 22))

# Extract unique dates from the filtered DataFrame
unique_dates = filtered_temp.select("date").distinct()

# Count the number of unique dates
num_dates = unique_dates.count()

# Show the unique dates and the count
print("Αριθμός Ημερών με θερμοκρασίες μεταξύ 18 και 22 C:", num_dates)

#print("Συγκεκριμένα αυτές οι ημερομινίες είχαν θερμοκρασία μεταξύ 18 και 22 C:")
#unique_dates.show()

Αριθμός Ημερών με θερμοκρασίες μεταξύ 18 και 22 C: 26


# Ερώτημα 1.2

##### Παραδοχή : Όταν λεει "10 πιο κρύες μέρες και οι 10 πιο ζεστές μέρες" θα μπορούσε ήτε να εννοεί "10 μερες που περιηχαν την πιο θερμη θερμοκρασια και 10 μερες που περιηχαν την πιο ψυχρή" αλλα λογικα εννοει "10 μερες με τις ψηλοτερες avg θερμοκρασίες" μιας και μονο αυτο βγαζει νοημα στον πραγματικό κόσμο. Οπότε βρήκα τους μεσους ορους καθε μερας παρακάτω :

In [101]:
# Filter out rows with null temperatures
tempm_data_filtered = tempm_data.filter(col("temperature").isNotNull())

# Aggregate temperatures by date
daily_avg_temps = tempm_data_filtered.groupBy("date").agg(avg("temperature").alias("avg_temperature"))

# Sort the DataFrame in descending order to get the warmest days
top_warmest_days = daily_avg_temps.orderBy(col("avg_temperature").desc()).limit(10)

# Sort the DatafRAM IN ASCENDING ORDER TO GET THE COLDEST DAYS
top_coldest_days = daily_avg_temps.orderBy(col("avg_temperature").asc()).limit(10)


# Show the top 10 warmest days
print("Οι 10 ημέρες με την υψηλότερη μέση θερμοκρασία:")
top_warmest_days.show()

# Show the top 10 coldest days
print("Οι 10 ημέρες με την χαμηλότερη μέση θερμοκρασία:")
top_coldest_days.show()

Οι 10 ημέρες με την υψηλότερη μέση θερμοκρασία:
+----------+------------------+
|      date|   avg_temperature|
+----------+------------------+
|2014-05-22|              20.0|
|2014-05-21| 17.63888888888889|
|2014-06-08| 17.63888888888889|
|2014-05-23|16.257142857142856|
|2014-06-02|16.056338028169016|
|2014-05-24|15.833333333333334|
|2014-06-03|15.830985915492958|
|2014-06-04|15.710144927536232|
|2014-06-07|15.666666666666666|
|2014-05-25|15.619718309859154|
+----------+------------------+

Οι 10 ημέρες με την χαμηλότερη μέση θερμοκρασία:
+----------+------------------+
|      date|   avg_temperature|
+----------+------------------+
|2014-03-27|2.8260869565217392|
|2014-02-14| 3.391304347826087|
|2014-03-02| 3.857142857142857|
|2014-03-11|3.9027777777777777|
|2014-03-01| 4.083333333333333|
|2014-02-13|               4.1|
|2014-03-03| 4.132075471698113|
|2014-03-25| 4.152777777777778|
|2014-03-05| 4.211267605633803|
|2014-02-17| 4.211267605633803|
+----------+------------------+



# Ερώτημα 1.3

In [114]:
# Αριθμός Μήνα σε Ελληνική Λέξη
def convert_to_greek_month(number):
    return {1: 'Ιανουάριος', 2: 'Φεβρουάριος', 3: 'Μάρτιος', 4: 'Απρίλιος', 5: 'Μάιος', 6: 'Ιούνιος', 7: 'Ιούλιος', 8: 'Αύγουστος', 9: 'Σεπτέμβριος', 10: 'Οκτώβριος', 11: 'Νοέμβριος', 12: 'Δεκέμβριος'}.get(number, "Invalid month number")

In [116]:
# Κάνω cast τον μήνα απο το timestamp
humidity_data_filtered = hum_data.withColumn("month", month("timestamp"))

# Χρησιμοποιώ την εντολή stddev για να βρω το standard deviation = τυπικη απόκλιση της υγρασίας ανά μήνα
monthly_stddev_humidity = humidity_data_filtered.groupBy("month").agg(stddev("humidity").alias("stddev_humidity"))
print("Τυπική απόκλιση της υγρασίας ανά μήνα:")
monthly_stddev_humidity.show()
# Find the month with the highest standard deviation
max_stddev_month = monthly_stddev_humidity.orderBy(col("stddev_humidity").desc()).first()["month"]

print(f"Όπως βλέπουμε , ο μήνας με την υψηλότερη τιπική απόκλιση είναι ο : {convert_to_greek_month(max_stddev_month)} (ο {max_stddev_month}ος μήνας)")

Τυπική απόκλιση της υγρασίας ανά μήνα:
+-----+------------------+
|month|   stddev_humidity|
+-----+------------------+
|    6|15.060744059108561|
|    3|15.234461545276856|
|    5|15.967352950493565|
|    4|17.733347100625892|
|    2| 9.973405675315439|
+-----+------------------+

Όπως βλέπουμε , ο μήνας με την υψηλότερη τιπική απόκλιση είναι ο : Απρίλιος (ο 4ος μήνας)
